### Implementing RNN

In [48]:
import pandas as pd
import numpy as np

In [49]:
df = pd.read_csv("../Datasets/df_over_14.csv")
df = df.drop(['Unnamed: 0'],axis=1)

### Initially we will calculate the days remaining in the hospital which is given by the hospital length of stay - the day in the hospital

In [50]:
df['days_remaining'] = df['hospital_length_of_stay']-df['day']
df = df.drop(['hospital_length_of_stay','day'],axis=1)

df.head(30)

,parent_id,systolic_blood_pressure,diastolic_blood_pressure,heart_rate,respiratory_rate,oxygen_saturation,temperature,highest_mean_arterial_pressure,lowest_mean_arterial_pressure,highest_heart_rate,...,Bilateral Ground Glass,Cardiomegaly,Edema,Effusion,Unilateral Consolidation,Bilateral Ground Glass Opacities,Bilateral consolidationinfiltration,Pulmonary Embolism,Scarring or Fibrosis,days_remaining
0,6,127.000000,76.000000,68.0,19.0,95.0,36.6,92.000000,80.000000,77.0,...,0,0,0,0,0,0,0,0,0,31
1,6,97.000000,60.000000,68.0,22.0,98.0,36.4,71.000000,67.000000,77.0,...,0,0,0,0,0,0,0,0,0,30
2,6,140.000000,68.000000,72.0,22.0,99.0,36.5,84.000000,84.000000,97.0,...,0,0,0,0,0,0,0,0,0,29
3,6,108.000000,63.000000,98.0,22.0,95.0,36.5,77.000000,77.000000,107.0,...,0,0,0,1,0,0,0,0,0,28
4,6,126.000000,77.000000,68.0,24.0,98.0,36.5,92.000000,92.000000,77.0,...,0,0,0,0,0,1,1,0,0,27
5,6,120.000000,82.000000,76.0,24.0,93.0,36.6,93.000000,93.000000,82.0,...,0,0,0,0,0,0,0,0,0,26
6,6,128.000000,62.000000,73.0,26.0,93.0,36.8,93.000000,81.000000,78.0,...,0,0,0,0,0,0,0,0,0,25
7,6,116.000000,74.000000,82.0,20.0,96.0,37.6,86.000000,66.000000,94.0,...,0,0,0,0,0,0,0,0,0,18
8,14,107.000000,64.000000,60.0,20.0,94.0,36.9,0.000000,0.000000,64.0,...,0,0,0,0,0,0,0,0,0,32
9,14,153.000000,72.000000,56.0,18.0,93.0,36.7,0.000000,0.000000,83.0,...,0,0,0,0,0,0,0,0,0,31


### We will scale the columns which arent encoded and have continious values

In [51]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

columns_binary = [
    'intubated', 'cardiac_arrest', 'arrested_time', 'major_cardiac_events', 
    'clinically_diagnosed_infections', 'mechanical_ventilation', 'antiarrhythmic_therapies', 
    'renal_replacement_therapy_dialysis', 'cardiovascular_mechanical_support', 'echocardiogram', 
    'chest_x_ray', 'chest_ct', 'head_ct', 'antimicrobial', 'anticoagulation', 'steroid',
    'Bilateral Consolidation', 'Bilateral Ground Glass', 'Cardiomegaly', 'Edema', 'Effusion', 
    'Pneumothorax', 'Unilateral Consolidation', 'Unilateral Ground Glass', 'Bilateral Ground Glass Opacities',
    'Bilateral consolidationinfiltration', 'Subarachnoid Hemorrhage', 'Subdural Hemorrhage',
    'Emphysematous or Bronchiectasis changes', 'Emphysematous or Bronchiectatic changes', 
    'Pulmonary Embolism', 'Scarring or Fibrosis', 'Unilateral Ground Glass Opacities', 
    'Unilateral consolidationinfiltration'
]

# Define your columns to exclude from scaling
columns_to_exclude = ['parent_id', 'days_remaining'] + columns_binary

# Select the columns to scale
columns_to_scale = [col for col in df.columns if col not in columns_to_exclude]

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform only the columns that need scaling
df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])

# Check the scaled data
df.head()


,parent_id,systolic_blood_pressure,diastolic_blood_pressure,heart_rate,respiratory_rate,oxygen_saturation,temperature,highest_mean_arterial_pressure,lowest_mean_arterial_pressure,highest_heart_rate,...,Bilateral Ground Glass,Cardiomegaly,Edema,Effusion,Unilateral Consolidation,Bilateral Ground Glass Opacities,Bilateral consolidationinfiltration,Pulmonary Embolism,Scarring or Fibrosis,days_remaining
0,6,0.001484,0.471299,-0.599641,-0.426718,0.274956,-0.415526,0.775369,1.002965,-0.781388,...,0,0,0,0,0,0,0,0,0,31
1,6,-1.416503,-0.741557,-0.599641,0.216288,1.190958,-0.847667,0.355284,0.668960,-0.781388,...,0,0,0,0,0,0,0,0,0,30
2,6,0.615945,-0.135129,-0.352470,0.216288,1.496292,-0.631597,0.615336,1.105736,0.443514,...,0,0,0,0,0,0,0,0,0,29
3,6,-0.896574,-0.514146,1.254138,0.216288,0.274956,-0.631597,0.475308,0.925887,1.055964,...,0,0,0,1,0,0,0,0,0,28
4,6,-0.045782,0.547103,-0.599641,0.644958,1.190958,-0.631597,0.775369,1.311277,-0.781388,...,0,0,0,0,0,1,1,0,0,27


### Define a simple RNN model

In [53]:
import torch
import torch.nn as nn
import torch.optim as optim

# This line defines a class SimpleRNN that inherits from torch.nn.Module. 
#In PyTorch, the Module class is the base class for all neural network modules,
#and it provides essential functions for building and training models.
class SimpleRNN(nn.Module):
    ## constructor that initiaes once the class SimpleRNN is called
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        ## this is an RNN layer that takes inputsize, hiddensize
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        ## this is a fully connected layer which will give us the output
        self.fc = nn.Linear(hidden_size, output_size)

   ## this is the way in which our RNN will operate in forward
    def forward(self, x):
        ## first the input values will be sent to the RNN
        ## it will give out two values out: This is the output of the RNN layer. 
            # It contains the hidden states of the RNN at all time steps.
        ##_: The second value returned by self.rnn(x) is the hidden state for the next time step
            #(which is usually not needed in a basic RNN setup like this, so we use _ to ignore it).
        out, _ = self.rnn(x)
        ## the out consists of the output from each of the timestamp 
        ## which is also the hidden layer for the next timestamp
        ## out[:, -1, :] gives the ouptut of the last layer 
        ## which is also the hidden layer of the next timestamp
        ## but in this case it will only be used to produce the output
        ## _ in the above gives the hidden state produced at timestamp t for the next timestamp (t+1)
        ## it is not used so discarded
        ## out[:,-1,:] and _ will give the same output
        out = out[:, -1, :] 
        ## below we will pass the output of the last timestep and pass to a fully connected layer
        out = self.fc(out)
        return out

### Function to extract the dataset for one patient_id

In [54]:
# Define the data extraction function for each patient
def get_data_for_parent(df, parent_id):
    data = df[df['parent_id'] == parent_id]
    features = data.drop(columns=['parent_id', 'days_remaining'])
    target = data['days_remaining'].values
    features_tensor = torch.tensor(features.values).float().unsqueeze(0)  # Add batch dimension
    target_tensor = torch.tensor(target).float().unsqueeze(0)  # Add batch dimension
    return features_tensor, target_tensor

In [56]:
import time

# Initialize the RNN model parameters
input_size = 65  # Number of input features
hidden_size = 64
output_size = 1  # We are predicting a single value (e.g., days_remaining)
num_epochs = 200  # Number of epochs

# Define the loss criterion and optimizer
criterion = nn.MSELoss()

# Save the models
models = []

# Record the start time
start_time = time.time()

# Loop through each parent_id and train the model
unique_parent_ids = df['parent_id'].unique()

for parent_id in unique_parent_ids:
    print(f"Training for parent_id {parent_id}...")
    
    # Get the data for the current parent_id
    features, target = get_data_for_parent(df, parent_id)
    
    # Initialize a new model and optimizer for each parent_id
    model = SimpleRNN(input_size, hidden_size, output_size)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Training loop for each parent_id
    model.train()  # Ensure model is in training mode
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        
        # Forward pass
        output = model(features)
        
        # Compute the loss
        loss = criterion(output, target)
        print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {loss.item()}")
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
    
    # Save the trained model
    model_path = f"Models/model_parent_{parent_id}.pth"
    torch.save(model.state_dict(), model_path)
    models.append(model)  # Add model to the list

    # Optionally, evaluate the model or predict on test data after training
    model.eval()  # Switch to evaluation mode
    with torch.no_grad():
        predictions = model(features)
        print(f"Predicted days_remaining for parent_id {parent_id}: {predictions.item()}")

# Record the end time
end_time = time.time()

# Calculate and print the total training time
total_time = end_time - start_time
print(f"Total training time: {total_time:.2f} seconds")

Training for parent_id 6...
Epoch [1/200] Loss: 717.6364135742188
Epoch [2/200] Loss: 701.8953247070312
Epoch [3/200] Loss: 686.76611328125
Epoch [4/200] Loss: 672.5366821289062
Epoch [5/200] Loss: 659.3148193359375
Epoch [6/200] Loss: 647.0418090820312
Epoch [7/200] Loss: 635.591552734375
Epoch [8/200] Loss: 624.823486328125
Epoch [9/200] Loss: 614.61572265625
Epoch [10/200] Loss: 604.8864135742188
Epoch [11/200] Loss: 595.5907592773438
Epoch [12/200] Loss: 586.7081298828125
Epoch [13/200] Loss: 578.2283935546875
Epoch [14/200] Loss: 570.144287109375
Epoch [15/200] Loss: 562.4466552734375
Epoch [16/200] Loss: 555.1224365234375
Epoch [17/200] Loss: 548.1543579101562
Epoch [18/200] Loss: 541.5216064453125
Epoch [19/200] Loss: 535.2023315429688
Epoch [20/200] Loss: 529.1746826171875
Epoch [21/200] Loss: 523.41796875
Epoch [22/200] Loss: 517.9137573242188
Epoch [23/200] Loss: 512.6459350585938
Epoch [24/200] Loss: 507.6014404296875
Epoch [25/200] Loss: 502.76922607421875
Epoch [26/200] Lo

/Users/asheshlalshrestha/anaconda3/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 8])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/asheshlalshrestha/anaconda3/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 8])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [120/200] Loss: 275.95654296875
Epoch [121/200] Loss: 274.0419616699219
Epoch [122/200] Loss: 272.1395263671875
Epoch [123/200] Loss: 270.24920654296875
Epoch [124/200] Loss: 268.37078857421875
Epoch [125/200] Loss: 266.5043640136719
Epoch [126/200] Loss: 264.649658203125
Epoch [127/200] Loss: 262.8067626953125
Epoch [128/200] Loss: 260.9754638671875
Epoch [129/200] Loss: 259.1557312011719
Epoch [130/200] Loss: 257.3475646972656
Epoch [131/200] Loss: 255.55079650878906
Epoch [132/200] Loss: 253.765380859375
Epoch [133/200] Loss: 251.99127197265625
Epoch [134/200] Loss: 250.228271484375
Epoch [135/200] Loss: 248.47654724121094
Epoch [136/200] Loss: 246.7357940673828
Epoch [137/200] Loss: 245.00607299804688
Epoch [138/200] Loss: 243.2872772216797
Epoch [139/200] Loss: 241.57931518554688
Epoch [140/200] Loss: 239.8821563720703
Epoch [141/200] Loss: 238.19566345214844
Epoch [142/200] Loss: 236.51995849609375
Epoch [143/200] Loss: 234.854736328125
Epoch [144/200] Loss: 233.20011901855

/Users/asheshlalshrestha/anaconda3/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 8])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/asheshlalshrestha/anaconda3/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 8])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [44/200] Loss: 3734.28076171875
Epoch [45/200] Loss: 3724.8193359375
Epoch [46/200] Loss: 3715.387451171875
Epoch [47/200] Loss: 3705.9833984375
Epoch [48/200] Loss: 3696.6083984375
Epoch [49/200] Loss: 3687.262451171875
Epoch [50/200] Loss: 3677.946044921875
Epoch [51/200] Loss: 3668.6611328125
Epoch [52/200] Loss: 3659.408203125
Epoch [53/200] Loss: 3650.189208984375
Epoch [54/200] Loss: 3641.005615234375
Epoch [55/200] Loss: 3631.8583984375
Epoch [56/200] Loss: 3622.748046875
Epoch [57/200] Loss: 3613.67578125
Epoch [58/200] Loss: 3604.64208984375
Epoch [59/200] Loss: 3595.646484375
Epoch [60/200] Loss: 3586.690185546875
Epoch [61/200] Loss: 3577.77197265625
Epoch [62/200] Loss: 3568.8916015625
Epoch [63/200] Loss: 3560.048828125
Epoch [64/200] Loss: 3551.242919921875
Epoch [65/200] Loss: 3542.47412109375
Epoch [66/200] Loss: 3533.74072265625
Epoch [67/200] Loss: 3525.04248046875
Epoch [68/200] Loss: 3516.37890625
Epoch [69/200] Loss: 3507.75
Epoch [70/200] Loss: 3499.15454101

...

/Users/asheshlalshrestha/anaconda3/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 8])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/asheshlalshrestha/anaconda3/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 8])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [140/200] Loss: 205.19833374023438
Epoch [141/200] Loss: 203.63766479492188
Epoch [142/200] Loss: 202.0882110595703
Epoch [143/200] Loss: 200.54986572265625
Epoch [144/200] Loss: 199.02255249023438
Epoch [145/200] Loss: 197.5061798095703
Epoch [146/200] Loss: 196.00059509277344
Epoch [147/200] Loss: 194.50582885742188
Epoch [148/200] Loss: 193.0217742919922
Epoch [149/200] Loss: 191.5482635498047
Epoch [150/200] Loss: 190.08526611328125
Epoch [151/200] Loss: 188.63278198242188
Epoch [152/200] Loss: 187.19065856933594
Epoch [153/200] Loss: 185.7587890625
Epoch [154/200] Loss: 184.33712768554688
Epoch [155/200] Loss: 182.9256591796875
Epoch [156/200] Loss: 181.52423095703125
Epoch [157/200] Loss: 180.13287353515625
Epoch [158/200] Loss: 178.75132751464844
Epoch [159/200] Loss: 177.37973022460938
Epoch [160/200] Loss: 176.01792907714844
Epoch [161/200] Loss: 174.6658172607422
Epoch [162/200] Loss: 173.32339477539062
Epoch [163/200] Loss: 171.9905242919922
Epoch [164/200] Loss: 170.6

/Users/asheshlalshrestha/anaconda3/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 8])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/asheshlalshrestha/anaconda3/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 8])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [71/200] Loss: 191.53445434570312
Epoch [72/200] Loss: 189.71005249023438
Epoch [73/200] Loss: 187.90158081054688
Epoch [74/200] Loss: 186.10903930664062
Epoch [75/200] Loss: 184.33241271972656
Epoch [76/200] Loss: 182.5717010498047
Epoch [77/200] Loss: 180.82687377929688
Epoch [78/200] Loss: 179.0980224609375
Epoch [79/200] Loss: 177.385009765625
Epoch [80/200] Loss: 175.68780517578125
Epoch [81/200] Loss: 174.00633239746094
Epoch [82/200] Loss: 172.34066772460938
Epoch [83/200] Loss: 170.69064331054688
Epoch [84/200] Loss: 169.05625915527344
Epoch [85/200] Loss: 167.4373321533203
Epoch [86/200] Loss: 165.83383178710938
Epoch [87/200] Loss: 164.24563598632812
Epoch [88/200] Loss: 162.67266845703125
Epoch [89/200] Loss: 161.11477661132812
Epoch [90/200] Loss: 159.5718231201172
Epoch [91/200] Loss: 158.04371643066406
Epoch [92/200] Loss: 156.5303192138672
Epoch [93/200] Loss: 155.0314483642578
Epoch [94/200] Loss: 153.54708862304688
Epoch [95/200] Loss: 152.0769500732422
Epoch [96

Epoch [189/200] Loss: 15.604547500610352
Epoch [190/200] Loss: 15.566810607910156
Epoch [191/200] Loss: 15.530168533325195
Epoch [192/200] Loss: 15.49460506439209
Epoch [193/200] Loss: 15.460081100463867
Epoch [194/200] Loss: 15.4265775680542
Epoch [195/200] Loss: 15.39406681060791
Epoch [196/200] Loss: 15.362520217895508
Epoch [197/200] Loss: 15.331918716430664
Epoch [198/200] Loss: 15.302230834960938
Epoch [199/200] Loss: 15.273438453674316
Epoch [200/200] Loss: 15.245511054992676
Predicted days_remaining for parent_id 511: 11.866296768188477
Total training time: 12.54 seconds


### Function for prediction through ensemble method as it takes the average of the output of all the models

In [57]:
# Load each model and create the ensemble prediction function
def ensemble_prediction(models, input_data):
    predictions = []
    
    # Get predictions from each model
    for model in models:
        model.eval()  # Switch to evaluation mode
        with torch.no_grad():
            output = model(input_data)
            predictions.append(output.item())  # Store the prediction
        
    # Average the predictions to form the final prediction
    return sum(predictions) / len(predictions)

### Testing in random 25% of the dataset

In [39]:
import time
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Preparing the test data
test_data = df.drop("parent_id", axis=1)
X = test_data.drop("days_remaining", axis=1)
y = test_data.days_remaining
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Initialize lists for true and predicted values
y_true = []
y_pred = []

# Record the start time
start_time = time.time()

# Evaluate ensemble model on the test set
for i in range(len(X_test)):
    # Prepare each test sample with the correct dimensions
    test_sample = torch.tensor(X_test.iloc[i].values).float().unsqueeze(0).unsqueeze(0)  # Shape (1, 1, input_size)
    true_value = y_test.iloc[i]  # Actual value for the test sample
    
    # Get the ensemble prediction
    prediction = ensemble_prediction(models, test_sample)
    
    # Append to lists for MSE calculation
    y_true.append(true_value)
    y_pred.append(prediction)

# Record the end time
end_time = time.time()

# Calculate Mean Squared Error
mse = mean_squared_error(y_true, y_pred)
print(f"Ensemble Model MSE on test data: {mse}")

# Calculate and print the total evaluation time
total_time = end_time - start_time
print(f"Total evaluation time: {total_time:.2f} seconds")

Ensemble Model MSE on test data: 689.4647915017564
Total evaluation time: 0.87 seconds


In [40]:
y_true

[18,
 8,
 17,
 11,
 15,
 13,
 38,
 15,
 23,
 15,
 14,
 20,
 53,
 13,
 23,
 12,
 14,
 14,
 11,
 11,
 15,
 24,
 16,
 15,
 20,
 15,
 27,
 37,
 23,
 3,
 10,
 12,
 37,
 9,
 18,
 40,
 21,
 15,
 8,
 8,
 46,
 13,
 15,
 10,
 14,
 14,
 80,
 21,
 17,
 24,
 17,
 14,
 36,
 12,
 21,
 9,
 20,
 7,
 32,
 28,
 9,
 65,
 27,
 15,
 58,
 11,
 15,
 10,
 52,
 55,
 21,
 14,
 20,
 11,
 34,
 5,
 9,
 75,
 20,
 21,
 31,
 25,
 16,
 15,
 34,
 28,
 54,
 28,
 20,
 1,
 33,
 17,
 14,
 30,
 14,
 19,
 32,
 36,
 19,
 66,
 13,
 12,
 16,
 8,
 15,
 43,
 17,
 15,
 31,
 7,
 22,
 20,
 25,
 5,
 18,
 13,
 23,
 30,
 22,
 22,
 58,
 18,
 13,
 24,
 39,
 43,
 34,
 27,
 25,
 27,
 30,
 15,
 28,
 18,
 33,
 11,
 39,
 30,
 32,
 29,
 22,
 13,
 31,
 28,
 26,
 37,
 18,
 48,
 5,
 42,
 9,
 39,
 9,
 33,
 1,
 11,
 8,
 33,
 11,
 14,
 24,
 56,
 19,
 40,
 50,
 23,
 26,
 40,
 26,
 33,
 19,
 28,
 42,
 12,
 41,
 8,
 2,
 14,
 17,
 12,
 15,
 10,
 39,
 22,
 9,
 12,
 36,
 13,
 15,
 11,
 16,
 48,
 36,
 17,
 3,
 20]

In [41]:
y_pred

[-0.13185802524035073,
 2.0282793961921515,
 -0.2022667840427282,
 -2.410910342154758,
 1.7636443002871713,
 -0.4671705573303055,
 0.2032238860063407,
 1.094239984543956,
 -1.6728887531374181,
 0.6553400960199687,
 2.2925445119641266,
 -0.40621846997920347,
 0.721347966630544,
 0.7624812117986837,
 1.5131583982432375,
 2.111413592624725,
 2.130959064680703,
 1.3697353292788779,
 2.409967783344339,
 1.6830194083859726,
 1.1615173786080308,
 -0.07296764090353129,
 1.7075479589972873,
 1.8583166660094748,
 1.04803083983383,
 0.008919557594523137,
 -0.3023595061077147,
 1.838167242477743,
 1.477879485153422,
 1.6193178690475774,
 1.791738170028037,
 1.1853421006199656,
 -0.3500823441664783,
 2.4162441260200374,
 1.646983121426738,
 1.8541254625674717,
 0.03546348590479822,
 2.0104839426788446,
 1.8543693803402843,
 1.4291005909290848,
 1.6393294780686194,
 0.9172725901767915,
 0.6132920407975206,
 2.480506772365497,
 0.5661859580942866,
 0.5611791121868455,
 0.9284347135345546,
 2.09677329

### Testing

In [ ]:
seq_len = 8
hidden_size = 64
batch_size = 1  # Single sequence in the batch

# Random sequence data of shape [1, 8, 80] (1 sequence, 8 time steps, 80 features)
sequence_data = torch.randn(batch_size, seq_len, 80)

# RNN model (hidden size 64)
rnn = nn.RNN(input_size=80, hidden_size=hidden_size, batch_first=True)

# Forward pass through the RNN
out, h_n = rnn(sequence_data)
print(out.shape)  # Output shape: [1, 8, 64] (1 batch, 8 time steps, 64 hidden units)

In [ ]:
last_output = out[:, -1, :]  # Shape: [1, 64], which is the hidden state of the last time step
last_output

In [ ]:
h_n[-1]

In [ ]:
h_n

In [18]:
data = df[df['parent_id'] == 3]
features = data.drop(columns=['parent_id', 'days_remaining'])
target = data['days_remaining'].values


(8, 65)